In [2]:
import json
import pandas as pd

data = pd.read_json('./train.json')
print(len(data))
print(data.head(3))

9397
   岗位ID                                               解析结果  简历ID
0   519  {'profileEduExps': [{'schoolLevel': [], 'schoo...     0
1   418  {'profileEduExps': [{'schoolLevel': ['14844587...     1
2   516  {'profileEduExps': [{'schoolLevel': ['62 63 50...     2


In [13]:
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

data = pd.read_json('./train.json')
data['解析结果'] = data['解析结果'].apply(lambda x : json.dumps(x).replace('"', ' ').replace('"', ' ').split())

train_data, test_data = train_test_split(data, train_size=0.75)

joblist = pd.read_json('./job_list.json')
joblist['解析结果'] = joblist['岗位名称'] + ' ' + joblist['岗位介绍'] + ' ' + joblist['岗位要求']
joblist['解析结果'] = joblist['解析结果'].apply(lambda x : x.split())



In [14]:
train_feat = []
for row in train_data.iterrows():
    label = row[1]['岗位ID']
    query_text= row[1]['解析结果']
    feat = [
        label,
        len(query_text), len(set(query_text)), len(query_text) - len(set(query_text)),
    ]
    for target_text in joblist['解析结果']:
        feat += [
            len(set(query_text) & set(target_text)),
            len(set(query_text) & set(target_text)) / len(query_text),
            len(set(query_text) & set(target_text)) / len(target_text),
            
            len(set(query_text) & set(target_text)) / len(set(target_text)),
            len(set(query_text) & set(target_text)) / len(set(query_text))

        ]
    train_feat.append(feat)
train_feat = np.array(train_feat)
m = RandomForestClassifier()
m.fit(
    train_feat[:, 1:],
    train_feat[:, 0],
)

test_feat = []
labels = []
for row in test_data.iterrows():
    labels.append(row[1]['岗位ID'])
    query_text= row[1]['解析结果']
    feat = [
        len(query_text), len(set(query_text)), len(query_text) - len(set(query_text)),
    ]
    for target_text in joblist['解析结果']:
        feat += [
            len(set(query_text) & set(target_text)),
            len(set(query_text) & set(target_text)) / len(query_text),
            len(set(query_text) & set(target_text)) / len(target_text),
            
            len(set(query_text) & set(target_text)) / len(set(target_text)),
            len(set(query_text) & set(target_text)) / len(set(query_text))

        ]
    test_feat.append(feat)
test_feat = np.array(test_feat)
pd.DataFrame({
    '简历ID': range(len(test_data)),
    '岗位ID': m.predict(test_feat).astype(int)
}).to_csv('./result.csv', index=None)

pred = m.predict(test_feat).astype(int)
F1 = f1_score(labels,pred,average='macro')
print(F1)




0.20009075381998143
